In [1]:
en_nouns_path = "baselines/predictions/en/fasttext/predicted_nouns_fasttext_en_ranked.tsv"
en_verbs_path = "baselines/predictions/en/fasttext/predicted_verbs_fasttext_en_ranked.tsv"
public_verbs_path = "baselines/predictions/ru/fasttext/predicted_public_verbs_fasttext_efficient.tsv"
private_verbs_path = "baselines/predictions/predicted_private_verbs_fasttext_wiki_new.tsv"
private_nouns_path = "baselines/predictions/predicted_private_nouns_poincare.tsv"
public_nouns_path = "baselines/predictions/predicted_public_nouns_fasttext_wiki_new.tsv"
food_wordnet_path = "baselines/predictions/semeval/predicted_TAXI_food_wordnet_en_only_orphan_hch.tsv"

In [2]:
gs_en_nouns_path = "../semevals/semeval-2016-task-14/reader/nouns_en_new.tsv"
gs_en_verbs_path = "../semevals/semeval-2016-task-14/reader/verbs_en_new.tsv"
gs_public_verbs_path = "dataset/public/verbs_public_subgraphs.tsv"
gs_private_verbs_path = "dataset/private/verbs_private_subgraphs.tsv"
gs_private_nouns_path = "dataset/private/nouns_private_subgraphs.tsv"
gs_public_nouns_path = "dataset/public/nouns_public_subgraphs.tsv"
gs_food_wordnet_path = "semeval2016_task13/data/gs_taxo/EN/food_wordnet_en.taxo"

In [3]:
from scoring_program.utils import *

In [4]:
en_nouns = read_dataset(en_nouns_path)
en_verbs = read_dataset(en_verbs_path)
public_verbs = read_dataset(public_verbs_path)
private_verbs = read_dataset(private_verbs_path)
private_nouns = read_dataset(private_nouns_path)
public_nouns = read_dataset(public_nouns_path)
food_wordnet = read_dataset(food_wordnet_path)

In [5]:
gs_en_nouns = read_dataset(gs_en_nouns_path, lambda x: json.loads(x))
gs_en_verbs = read_dataset(gs_en_verbs_path, lambda x: json.loads(x))
gs_public_verbs = read_dataset(gs_public_verbs_path, lambda x: json.loads(x))
gs_private_verbs = read_dataset(gs_private_verbs_path, lambda x: json.loads(x))
gs_private_nouns = read_dataset(gs_private_nouns_path, lambda x: json.loads(x))
gs_public_nouns = read_dataset(gs_public_nouns_path, lambda x: json.loads(x))

In [6]:
gs_food_wordnet = {}
with open(gs_food_wordnet_path, 'r') as f:
    for line in f:
        _, word, hypernym = line.strip().split("\t")
        gs_food_wordnet[word] = [hypernym]

In [6]:

def get_score(true, predicted, k=10):
    ap_sum = 0
    rr_sum = 0

    for neologism in true:
        # getting sets of hypernyms for true and predicted
        true_hypernyms = true.get(neologism, [])
        predicted_hypernyms = predicted.get(neologism, [])

        # get metrics
        ap_sum += compute_ap(true_hypernyms, predicted_hypernyms, k)
        rr_sum += compute_rr(set([j for i in true_hypernyms for j in i]), predicted_hypernyms, k)
    return ap_sum / len(true), rr_sum / len(true)


def compute_ap(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0
    already_predicted = set()
    skipped = 0
    for i, p in enumerate(predicted):
        if p in already_predicted:
            skipped += 1
            continue
        for parents in actual:
            if p in parents:
                num_hits += 1.0
                score += num_hits / (i + 1.0 - skipped)
                already_predicted.update(parents)
                break

    return score / min(len(actual), k)


def compute_rr(true, predicted, k=10):
    for i, synset in enumerate(predicted[:k]):
        if synset in true:
            return 1.0 / (i + 1.0)
    return 0.0

In [12]:
def count_upper_bound(true, predicted):
    length = 0
    for group in true:
        if len(set(predicted).intersection(group)) > 0:
            length += 1
    return length/len(true)


sum([count_upper_bound(gs_private_nouns[i], private_nouns[i]) for i in gs_private_nouns]) / len(gs_private_nouns)

0.2674316939890711

In [7]:
print(get_score(gs_private_nouns, private_nouns))

(0.12462759129152577, 0.13606297163674214)


In [9]:
compute_ap(gs_public_nouns['АБДОМИНОПЛАСТИКА'], public_nouns['АБДОМИНОПЛАСТИКА'])

1.0

In [10]:
def compute_ap(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0
    already_predicted = set()
    skipped = 0
    for i, p in enumerate(predicted):
        if p in already_predicted:
            skipped += 1
            continue
        prev_num_hits = num_hits
        for parents in actual:
            if p in parents:
                num_hits += 1.0
                score += num_hits / (i + 1.0 - skipped)
                already_predicted.update(parents)
                break
        if prev_num_hits == num_hits:
            skipped += 1
    return score / min(len(actual), k)

In [11]:
print(get_score(gs_en_nouns, en_nouns))
print(get_score(gs_en_verbs, en_verbs))
print(get_score(gs_public_nouns, public_nouns))
print(get_score(gs_private_nouns, private_nouns))
print(get_score(gs_public_verbs, public_verbs))
print(get_score(gs_private_verbs, private_verbs))

(0.561353598691385, 0.4052562704471109)
(0.3290155440414508, 0.1947302409737643)
(0.7644356955380578, 0.5913469149689616)
(0.7583278688524588, 0.5915448868071831)
(0.4938095238095238, 0.3270657596371885)
(0.5907142857142859, 0.4474784580498865)


In [12]:
from ruwordnet.ruwordnet_reader import RuWordnet

In [13]:
rwn = RuWordnet("dataset/ruwordnet.db", None)

In [15]:
wiki_ru = {}
with open("wiki_ru.jsonlines", 'r') as f:
    for line in f:
        line = json.loads(line)
        wiki_ru[line['word']] = {"hypernyms": line['hypernyms'], "meanings": line['meanings']}

In [16]:
import re
delete_bracets = re.compile(r"\(.+?\)")
for word, candidates in private_nouns.items():
    print([[rwn.get_name_by_id(j) for j in i] for i in gs_public_nouns[word]])
    for candidate in candidates:
        if word.lower() in wiki_ru:
            wiktionary_data = wiki_ru[word.lower()]
            candidate_word = rwn.get_name_by_id(candidate)
            candidate_words = delete_bracets.sub("", candidate_word).split(',')
            in_hypernyms = any([candidate_word.lower() in wiktionary_data['hypernyms'] for candidate_word in candidate_words])
            in_definitions = any([any([candidate_word.lower() in i for candidate_word in candidate_words]) for i in wiktionary_data['meanings']])
            in_true = any([candidate in i for i in gs_public_nouns[word]])
            print(word, candidate_word, in_hypernyms, in_definitions, in_true)
            #print(wiktionary_data['meanings'], wiktionary_data['hypernyms'])
    print("======")

[]
АБСЕНТЕИЗМ ПОЛИТИЧЕСКОЕ ТЕЧЕНИЕ False False False
АБСЕНТЕИЗМ МИРОВОЗЗРЕНИЕ False False False
АБСЕНТЕИЗМ УБЕЖДЕНИЯ, ВЗГЛЯДЫ False False False
АБСЕНТЕИЗМ ИДЕОЛОГИЯ False False False
АБСЕНТЕИЗМ ФИЛОСОФСКОЕ УЧЕНИЕ False False False
АБСЕНТЕИЗМ ПРИСПОСОБЛЕНЧЕСТВО False False False
АБСЕНТЕИЗМ СКЕПТИЦИЗМ, СКЕПТИЧЕСКОЕ ОТНОШЕНИЕ False False False
АБСЕНТЕИЗМ ПОЛИТИЧЕСКОЕ НАПРАВЛЕНИЕ False False False
АБСЕНТЕИЗМ НАРКОЛОГИЧЕСКОЕ РАССТРОЙСТВО False False False
АБСЕНТЕИЗМ ПРАВАЯ ПОЛИТИЧЕСКАЯ ОРИЕНТАЦИЯ False False False
[]
АБСОЛЮТИЗАЦИЯ ИСКАЗИТЬ, ПРЕДСТАВИТЬ В ЛОЖНОМ СВЕТЕ False False False
АБСОЛЮТИЗАЦИЯ ПРЕУВЕЛИЧИТЬ False False False
АБСОЛЮТИЗАЦИЯ УМАЛИТЬ, ПРЕУМЕНЬШИТЬ False False False
АБСОЛЮТИЗАЦИЯ ПРОЦЕСС, СМЕНА СОСТОЯНИЙ False False False
АБСОЛЮТИЗАЦИЯ ОЦЕНИТЬ, ДАТЬ ОЦЕНКУ False False False
АБСОЛЮТИЗАЦИЯ ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ False False False
АБСОЛЮТИЗАЦИЯ СОЦИАЛЬНЫЙ ПРОЦЕСС False False False
АБСОЛЮТИЗАЦИЯ УМОЗРИТЕЛЬНЫЙ False False False
АБСОЛЮТИЗАЦИЯ ОШИБИТЬСЯ False False False
АБСОЛЮТИЗАЦ

АНДРОЛОГ МЕДИЦИНСКИЙ РАБОТНИК False False False
АНДРОЛОГ ЛЕКАРЬ, ЦЕЛИТЕЛЬ False False False
АНДРОЛОГ ТЕРАПЕВТ False False False
АНДРОЛОГ ХИРУРГ False False False
АНДРОЛОГ НЕВРОЛОГ False False False
АНДРОЛОГ РАДИОЛОГ False False False
АНДРОЛОГ ВЕТЕРИНАРНЫЙ РАБОТНИК False False False
АНДРОЛОГ ПСИХОЛОГ False False False
АНДРОЛОГ ФАРМАЦЕВТ False False False
[]
АНДРОЛОГИЯ МЕДИЦИНА True False False
АНДРОЛОГИЯ СФЕРА ДЕЯТЕЛЬНОСТИ False False False
АНДРОЛОГИЯ ХИРУРГИЯ False False False
АНДРОЛОГИЯ МЕДИЦИНСКАЯ НАУКА False False False
АНДРОЛОГИЯ РАДИАЦИОННАЯ МЕДИЦИНА False False False
АНДРОЛОГИЯ ДЕРМАТОВЕНЕРОЛОГИЯ False False False
АНДРОЛОГИЯ ПЕДАГОГИЧЕСКИЕ НАУКИ False False False
АНДРОЛОГИЯ БИОЛОГИЯ False False False
АНДРОЛОГИЯ ЕСТЕСТВЕННЫЕ НАУКИ False False False
АНДРОЛОГИЯ ПЛАСТИЧЕСКАЯ ХИРУРГИЯ False False False
[]
АНИМАЛИСТ ХУДОЖНИК (АВТОР ИЗОБРАЗИТЕЛЬНОГО ПРОИЗВЕДЕНИЯ) False True False
АНИМАЛИСТ ДЕЯТЕЛЬ ИСКУССТВА False False False
АНИМАЛИСТ ЖИВОПИСЕЦ False False False
АНИМАЛИСТ АВТОР False Fa

АРЬЕРГАРД РОД ВОЙСК False False False
[]
АСЕПТИКА САНИТАРНАЯ ОБРАБОТКА False False False
АСЕПТИКА ДЕЗИНФЕКЦИЯ False False False
АСЕПТИКА МЕДИЦИНА False False False
АСЕПТИКА ТЕРАПИЯ False False False
АСЕПТИКА МЕТОД ЛЕЧЕНИЯ False False False
АСЕПТИКА ТЕРМИЧЕСКАЯ ОБРАБОТКА False False False
АСЕПТИКА САНИТАРНЫЕ МЕРОПРИЯТИЯ False False False
АСЕПТИКА МИКРОБИОЛОГИЯ False False False
АСЕПТИКА БИОЛОГИЯ False False False
АСЕПТИКА МЕДИКАМЕНТОЗНАЯ ТЕРАПИЯ False False False
[]
АСКАРИДОЗ БОЛЕЗНЬ False False False
АСКАРИДОЗ ИНФЕКЦИОННАЯ БОЛЕЗНЬ False False False
АСКАРИДОЗ ЖЕЛУДОЧНО-КИШЕЧНЫЕ ЗАБОЛЕВАНИЯ False False False
АСКАРИДОЗ ВОСПАЛЕНИЕ (БОЛЕЗНЕННЫЙ ПРОЦЕСС) False False False
АСКАРИДОЗ ПАРАЗИТАРНАЯ БОЛЕЗНЬ False False False
АСКАРИДОЗ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ False False False
АСКАРИДОЗ БИОЛОГИЧЕСКИЙ ПАРАЗИТИЗМ False False False
АСКАРИДОЗ БОЛЕЗНЕТВОРНЫЙ МИКРООРГАНИЗМ False False False
АСКАРИДОЗ БИОЛОГИЧЕСКИЙ АГЕНТ False False False
АСКАРИДОЗ КОЖНАЯ БОЛЕЗНЬ False False False
[]
АСПАРТАМ

БАРАБАШКА ЖИВОТНОЕ False False False
БАРАБАШКА ПЛОХОЙ ЧЕЛОВЕК False False False
БАРАБАШКА ФИГУРА (ИЗОБРАЖЕНИЕ) False False False
[]
БАРБАРИС КУСТАРНИК False True False
БАРБАРИС РАСТЕНИЕ True True False
БАРБАРИС ЯГОДА True True False
БАРБАРИС ЯГОДНАЯ КУЛЬТУРА False False False
БАРБАРИС ТРАВЯНИСТОЕ РАСТЕНИЕ False False False
БАРБАРИС ВЕЧНОЗЕЛЕНОЕ РАСТЕНИЕ False False False
БАРБАРИС ПРЯНОСТЬ False False False
БАРБАРИС ДРЕВЕСНОЕ РАСТЕНИЕ False False False
БАРБАРИС МНОГОЛЕТНИЕ ТРАВЫ False False False
БАРБАРИС ДРЕВЕСНО-КУСТАРНИКОВАЯ ПОРОДА False False False
[]
БАРДА ОТХОДЫ True False False
БАРДА РУДА False False False
БАРДА ПОЛЕЗНЫЕ ИСКОПАЕМЫЕ False False False
БАРДА МЕТАЛЛУРГИЧЕСКОЕ СЫРЬЕ False False False
БАРДА НЕМЕТАЛЛЫ False False False
БАРДА МИНЕРАЛЬНО-СЫРЬЕВАЯ ПРОДУКЦИЯ False False False
БАРДА ГОРОД False True False
БАРДА ПЕРЕРАБОТКА ПИЩЕВОГО СЫРЬЯ False False False
БАРДА МИНЕРАЛЬНОЕ УДОБРЕНИЕ False False False
БАРДА ПЕСНЯ False False False
[]
БАРК НАДВОДНОЕ СУДНО False False False
БАР

БЕРЦЫ ВАЛЯНАЯ ОБУВЬ False False False
БЕРЦЫ ОДЕЖДА False False False
БЕРЦЫ ЗИМНЯЯ ОБУВЬ False False False
БЕРЦЫ СПОРТИВНАЯ ОБУВЬ False False False
БЕРЦЫ ДЕТАЛЬ ОБУВИ False False False
[]
БЕСКОЗЫРКА ГОЛОВНОЙ УБОР False False False
БЕСКОЗЫРКА ОДЕЖДА False False False
БЕСКОЗЫРКА ШЛЯПА False False False
БЕСКОЗЫРКА ШАПКА False False False
БЕСКОЗЫРКА ФОРМЕННАЯ ОДЕЖДА False False False
БЕСКОЗЫРКА РУБАШКА (ОДЕЖДА) False False False
БЕСКОЗЫРКА МУЖСКАЯ ОДЕЖДА False False False
БЕСКОЗЫРКА ЖЕНСКАЯ ОДЕЖДА False False False
БЕСКОЗЫРКА КОФТА False False False
БЕСКОЗЫРКА ПЛАТОК (ПРЕДМЕТ ОДЕЖДЫ) False False False
[]
БЕССРЕБРЕНИК ЧЕЛОВЕК True True False
БЕССРЕБРЕНИК ПЛОХОЙ ЧЕЛОВЕК False False False
БЕССРЕБРЕНИК БЕСКОРЫСТНЫЙ ЧЕЛОВЕК False True False
БЕССРЕБРЕНИК БОГАЧ False False False
БЕССРЕБРЕНИК ПРЕСТУПНИК False False False
БЕССРЕБРЕНИК ВЕРУЮЩИЙ False False False
БЕССРЕБРЕНИК ПРАВЕДНИК, ПРАВЕДНЫЙ ЧЕЛОВЕК False False False
БЕССРЕБРЕНИК ОБМАНЩИК False False False
БЕССРЕБРЕНИК ЛИЦЕМЕР False False False
Б

...

ФАЛЕРИСТИКА ФИЛОЛОГИЯ False False False
ФАЛЕРИСТИКА ПРИКЛАДНЫЕ НАУКИ False False False
ФАЛЕРИСТИКА ВОЕННОЕ СНАРЯЖЕНИЕ False False False
[]
ФАРИНГИТ ВОСПАЛЕНИЕ (БОЛЕЗНЕННЫЙ ПРОЦЕСС) False True False
ФАРИНГИТ БОЛЕЗНЬ False False False
ФАРИНГИТ БОЛЕЗНИ ДЫХАТЕЛЬНЫХ ПУТЕЙ False False False
ФАРИНГИТ ПРОСТУДНОЕ ЗАБОЛЕВАНИЕ False False False
ФАРИНГИТ БОЛЕЗНИ ОРГАНОВ ДЫХАНИЯ False False False
ФАРИНГИТ КЛИНИЧЕСКИЙ СИМПТОМ False False False
ФАРИНГИТ СИНУСИТ False False False
ФАРИНГИТ РЕСПИРАТОРНАЯ ИНФЕКЦИЯ False False False
ФАРИНГИТ ВИРУСНАЯ БОЛЕЗНЬ False False False
ФАРИНГИТ СИМПТОМ (ВНЕШНИЙ ПРИЗНАК) False False False
[]
ФАРМАКОПЕЯ МЕДИЦИНА False False False
ФАРМАКОПЕЯ ПРОМЫШЛЕННОСТЬ False False False
ФАРМАКОПЕЯ СФЕРА ДЕЯТЕЛЬНОСТИ False False False
ФАРМАКОПЕЯ ПЕРЕРАБАТЫВАЮЩАЯ ПРОМЫШЛЕННОСТЬ False False False
ФАРМАКОПЕЯ ЕСТЕСТВЕННЫЕ НАУКИ False False False
ФАРМАКОПЕЯ НАУКА False False False
ФАРМАКОПЕЯ МЕДИЦИНСКАЯ ПРОМЫШЛЕННОСТЬ False False False
ФАРМАКОПЕЯ РЕЦЕПТУРА False False False
ФАРМАКОПЕЯ П

ФОРШТЕВЕНЬ ПАРУСНАЯ МАЧТА False False False
[]
ФОСГЕН ГАЗООБРАЗНОЕ ВЕЩЕСТВО False False False
ФОСГЕН ВЕЩЕСТВО True True False
ФОСГЕН ЯДОВИТОЕ ВЕЩЕСТВО False False False
ФОСГЕН НЕМЕТАЛЛЫ False False False
ФОСГЕН БОЕВОЕ ОТРАВЛЯЮЩЕЕ ВЕЩЕСТВО False False False
ФОСГЕН ЯДОВИТЫЙ ГАЗ False False False
ФОСГЕН ЖИДКОСТЬ False False False
ФОСГЕН НЕРВНО-ПАРАЛИТИЧЕСКИЕ ВЕЩЕСТВА False False False
ФОСГЕН УГЛЕВОДОРОД False False False
ФОСГЕН ХИМИЧЕСКОЕ СОЕДИНЕНИЕ False False False
[]
ФОТОШОП ИЗОБРАЖЕНИЕ (РЕЗУЛЬТАТ) False False False
ФОТОШОП ФОТОГРАФИЧЕСКОЕ ИЗОБРАЖЕНИЕ False False False
ФОТОШОП ПОДРИСОВАТЬ, ПОДПРАВИТЬ РИСУНОК False False False
ФОТОШОП ПРОИЗВЕДЕНИЕ ИЗОБРАЗИТЕЛЬНОГО ИСКУССТВА False False False
ФОТОШОП ЦИФРОВАЯ ТЕХНИКА False False False
ФОТОШОП ЦИФРОВОЕ ИЗОБРАЖЕНИЕ False False False
ФОТОШОП МОНТАЖ ЗАПИСЕЙ, СНИМКОВ False False False
ФОТОШОП ИМИТИРОВАТЬ False False False
ФОТОШОП ПРИЕМ (СПОСОБ) False False False
ФОТОШОП ОБРАЗЕЦ ДЛЯ ПОДРАЖАНИЯ False False False
[]
ФРАЗЕОЛОГИЗМ ЯЗЫКОВОЕ ВЫРАЖЕН

ХЭШТЕГ МАССОВАЯ АКЦИЯ False False False
ХЭШТЕГ АКЦИЯ (ДЕЙСТВИЕ) False False False
ХЭШТЕГ СОЦИАЛЬНАЯ СЕТЬ (ИНТЕРНЕТ-ПОРТАЛ) False False False
ХЭШТЕГ ЛИСТ БУМАГИ False False False
ХЭШТЕГ ИЗОБРАЖЕНИЕ (РЕЗУЛЬТАТ) False False False
[]
ЦВЕТОМУЗЫКА ЗВУКОТЕХНИКА False False False
ЦВЕТОМУЗЫКА РАДИОЭЛЕКТРОННОЕ ОБОРУДОВАНИЕ False False False
ЦВЕТОМУЗЫКА МУЗЫКАЛЬНОЕ ИСКУССТВО False False False
ЦВЕТОМУЗЫКА ЭЛЕКТРОННОЕ ОБОРУДОВАНИЕ False False False
ЦВЕТОМУЗЫКА МУЗЫКАЛЬНОЕ ПРОИЗВЕДЕНИЕ False False False
ЦВЕТОМУЗЫКА КУЛЬТУРНО-ЗРЕЛИЩНОЕ МЕРОПРИЯТИЕ False False False
ЦВЕТОМУЗЫКА ЭЛЕКТРОТЕХНИЧЕСКОЕ ОБОРУДОВАНИЕ False False False
ЦВЕТОМУЗЫКА ИСКУССТВЕННОЕ ОСВЕЩЕНИЕ False False False
ЦВЕТОМУЗЫКА ИНЖЕНЕРНОЕ ОБОРУДОВАНИЕ False False False
ЦВЕТОМУЗЫКА ТЕХНИЧЕСКОЕ УСТРОЙСТВО False False False
[]
ЦЕЛЛУЛОИД МАТЕРИАЛ ДЛЯ ИЗГОТОВЛЕНИЯ False False False
ЦЕЛЛУЛОИД МАТЕРИАЛ, ТКАНЬ True True False
ЦЕЛЛУЛОИД СИНТЕТИЧЕСКИЕ МАТЕРИАЛЫ False False False
ЦЕЛЛУЛОИД ВЕЩЕСТВО False False False
ЦЕЛЛУЛОИД ПЛЕНКА (МАТЕРИАЛ) Fals

ЧИСТОТЕЛ РАСТЕНИЕ True True False
ЧИСТОТЕЛ СОРНОЕ РАСТЕНИЕ False False False
ЧИСТОТЕЛ ЦВЕТКОВОЕ РАСТЕНИЕ False False False
ЧИСТОТЕЛ КУСТАРНИК False False False
ЧИСТОТЕЛ ВЕЧНОЗЕЛЕНОЕ РАСТЕНИЕ False False False
ЧИСТОТЕЛ ЛУКОВИЧНОЕ РАСТЕНИЕ False False False
ЧИСТОТЕЛ СПОРОВОЕ РАСТЕНИЕ False False False
ЧИСТОТЕЛ ТРОПИЧЕСКОЕ РАСТЕНИЕ False False False
ЧИСТОТЕЛ МНОГОЛЕТНИЕ ТРАВЫ False False False
[]
ЧРЕВОВЕЩАТЕЛЬ АРТИСТ False False False
ЧРЕВОВЕЩАТЕЛЬ КОМИК (ТОТ, КТО СМЕШИТ) False False False
ЧРЕВОВЕЩАТЕЛЬ ЧЕЛОВЕК False False False
ЧРЕВОВЕЩАТЕЛЬ ЦИРКОВОЙ АРТИСТ False False False
ЧРЕВОВЕЩАТЕЛЬ ИСПОЛНИТЕЛЬ ПРОИЗВЕДЕНИЯ False False False
ЧРЕВОВЕЩАТЕЛЬ ДЕЯТЕЛЬ ИСКУССТВА False False False
ЧРЕВОВЕЩАТЕЛЬ ШУТНИК False False False
ЧРЕВОВЕЩАТЕЛЬ ВЕСЕЛЫЙ ЧЕЛОВЕК False False False
ЧРЕВОВЕЩАТЕЛЬ РАБОТНИК False False False
ЧРЕВОВЕЩАТЕЛЬ ЭСТРАДНЫЙ АРТИСТ False False False
[]
ЧУДО-ЮДО ЖИВОТНОЕ False False False
ЧУДО-ЮДО ЖИВОЙ ОРГАНИЗМ False False False
ЧУДО-ЮДО ПЕРСОНАЖ False True False
ЧУДО-ЮДО НАСЕКОМОЕ F

ЭКВАЙРИНГ ПРОГРАММНО-АППАРАТНЫЙ КОМПЛЕКС False False False
ЭКВАЙРИНГ ФИНАНСОВАЯ ДЕЯТЕЛЬНОСТЬ False False False
ЭКВАЙРИНГ ПРЕДПРИНИМАТЕЛЬСТВО False False False
ЭКВАЙРИНГ ЭКОНОМИЧЕСКАЯ ДЕЯТЕЛЬНОСТЬ False False False
[]
ЭКВАЛАЙЗЕР ПЛЕЙЕР False False False
ЭКВАЛАЙЗЕР ЗВУКОВОСПРОИЗВОДЯЩАЯ АППАРАТУРА False False False
ЭКВАЛАЙЗЕР БЫТОВАЯ ЭЛЕКТРОНИКА False False False
ЭКВАЛАЙЗЕР ЗВУК False False False
ЭКВАЛАЙЗЕР ТЕХНИЧЕСКОЕ УСТРОЙСТВО False False False
ЭКВАЛАЙЗЕР ЗВУКОТЕХНИКА False False False
ЭКВАЛАЙЗЕР ПРЕОБРАЗОВАТЕЛЬ (УСТРОЙСТВО) False False False
ЭКВАЛАЙЗЕР МУЗЫКАЛЬНЫЙ ИНСТРУМЕНТ False False False
ЭКВАЛАЙЗЕР УСИЛИТЕЛЬ, УСИЛИТЕЛЬНОЕ УСТРОЙСТВО False False False
ЭКВАЛАЙЗЕР ЭЛЕКТРОННОЕ ОБОРУДОВАНИЕ False False False
[]
ЭКЛЕР КОНДИТЕРСКОЕ ИЗДЕЛИЕ False False False
ЭКЛЕР ПЕЧЕНЫЕ ИЗДЕЛИЯ False False False
ЭКЛЕР СЛАДКОЕ КУШАНЬЕ False False False
ЭКЛЕР ПРОДУКТЫ ПИТАНИЯ False False False
ЭКЛЕР МУЧНЫЕ ПРОДУКТЫ False False False
ЭКЛЕР ПИРОГ False False False
ЭКЛЕР ПИРОЖОК False False False
ЭКЛЕР ПИРО

In [40]:
wiki_ru

{}

In [65]:
[[rwn.get_name_by_id(j) for j in i] for i in gs_public_nouns['АКВЕДУК']]

[['ИНЖЕНЕРНОЕ СООРУЖЕНИЕ'],
 ['ДОРОЖНОЕ СООРУЖЕНИЕ', 'МОСТ'],
 ['ВОДОХОЗЯЙСТВЕННОЕ СООРУЖЕНИЕ']]

In [110]:
wiki_ru['АЛЕБАРДА'.lower()]

{'hypernyms': ['холодное оружие', 'оружие'],
 'meanings': ['старинное холодное оружие в виде фигурного топорика на длинном древке оканчивающееся копьём  и протазанами секирами и бердышами с золотыми орлами на древках с кистями золотыми и серебряными дал каждому воину сверх поместья от 40 до 70 рублей денежного жалованья  и с того времени уже никуда не ездил и не ходил один всюду провождаемый сими грозными телохранителями за коими только вдали следовали бояре и царедворцы эти морские топоры при абордаже рубящие и канаты и человека с головы до пояса']}

In [ ]:
import re
delete_bracets = re.compile(r"\(.+?\)")
for word, candidates in public_nouns.items():